<img src = '../../sb_tight.png'>
<h1 align = 'center'> Capstone Project 2: Pump It Up </h1>

---

### Notebook 3: Pre-processing
**Author:<br>
Tashi T. Gurung**<br>
**hseb.tashi@gmail.com**

### About the project:
The **objective** of this project is to **predict the failure of water points** spread accross Tanzania before they occur.

50% of Tanzania's population do not have access to safe water. Among other sources, Tanzanians depend on water points mostly pumps (~60K) spread across Tanzania. Compared to other infrastructure projects, water point projects consist of a huge number of inspection points that are geographically spread out. Gathering data on the condition of these pumps has been a challenge. From working with local agencies, to implementing mobile based crowd sourcing projects, none have produced satisfactory results.

The lack of quality data creates a number of problem for a stakeholder like the Tanzanian Government, specifically the Ministry of Water. Consequences include not only higher maintainence costs, but also all the problems and nuanced issues faced by communities when their access to water is compromised or threatened.

While better data collection infrastructure should be built overtime, this project (with its model(s), various analysis, and insights) will be key for efficient resource allocation to maximize the number of people and communities with access to water.
In the long run, it will assist stake holders in and project planning, and even local, regional and national level policy formation. 

### About the notebook:
The data for our project exists in two separate datasets:
1. Containing potential features
2. Containing target variable

In this notebook, we combine these datasets.\
We also perform preliminary EDA, and look at duplicate values and missing values.

Finally, we export this combined dataset for further EDA.

There are columns where Null/NaN is represented by 0.

In [62]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

for col in df.select_dtypes(include=numerics).columns:
    print(col)
    print(round(df[col].value_counts(normalize= True).iloc[0] #assuming 0 and 0.0 are the first (0th) element
                *100,2), 
                f'% of {col} values is 0')

amount_tsh
70.1 % of amount_tsh values is 0
gps_height
34.41 % of gps_height values is 0
longitude
3.05 % of longitude values is 0
latitude
3.05 % of latitude values is 0
num_private
98.73 % of num_private values is 0
population
35.99 % of population values is 0
construction_year
6.84 % of construction_year values is 0


For column * **construction_year**, we can be certain that the 0s represent missing value.\
For column **population**, we are not sure.\
For other columns, we can be skeptical, but do not currently have enough information to arrive to such conclusions.

---

Drop Columns

In [66]:
df['num_private'].value_counts(normalize = True)

0      0.987256
6      0.001364
1      0.001229
5      0.000774
8      0.000774
         ...   
180    0.000017
213    0.000017
23     0.000017
55     0.000017
94     0.000017
Name: num_private, Length: 65, dtype: float64

In [67]:
cols_to_drop = ['recorded_by',  # only 1 unique value
                'num_private'] # ~99% of data is 0, also no info in data dictionary
df.drop(columns = cols_to_drop, inplace = True)